In [1]:
from CoRe_Dataloader_From_File_With_Random_From_Tensors import get_new_ttv_dataloaders as getData
from CoRe_Dataloader_ECSG import dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,TensorDataset
import math
import torchinfo
import time
import numpy as np
import wandb
import datetime
from collections import OrderedDict

In [2]:
# import vit
# import vit_pytorch
from vit_pytorch import vit_for_small_dataset as vit_sd
from vit_pytorch import vit as simple_vit
from vit_pytorch.deepvit import DeepViT
from vit_pytorch import MAE
def init_model():
    # return simple_vit.ViT(image_size=400,
    #                patch_size=20,
    #                num_classes=19,
    #                dim=int(1024/2),
    #                depth=2,
    #                heads=8,
    #                mlp_dim=int(2048/2),
    #                channels=1).to("cuda:0")
    # return vit_sd.ViT(image_size=400,
    #                patch_size=20,
    #                num_classes=19,
    #                dim=1024,
    #                depth=4,
    #                heads=16,
    #                mlp_dim=int(2048/2),
    #                dropout = 0.1,
    #                emb_dropout = 0,
    #                channels=1).to("cuda:0")
    return DeepViT(image_size=400,
                    patch_size=20,
                    num_classes=19,
                    dim=1024,
                    depth=4,
                    heads=16,
                    mlp_dim=int(2048/2),
                    dropout=0.1,
                    emb_dropout=0.1,
                    channels=1).to("cuda:0")


In [3]:
def tobtime(s):
    return f"{s//3600}H:{(s%3600)//60}M:{s%60}S"
def ismult(n,div):
    return bool(1>>(n%div))

In [5]:
encoder = init_model()
mae = MAE(
    encoder = encoder,
    masking_ratio = .25,
    decoder_dim=1024,
    decoder_depth=6,
).to("cuda:0")
data,_,_ = getData()    

In [6]:
for i, (a,_) in enumerate(data):
    a = a.unsqueeze(1).float().to("cuda:0")
    loss = mae(a)
    loss.backward()
    print(i, loss.item())
    

0 1.041434407234192
1 1.0393342971801758
2 1.040280818939209
3 1.0396443605422974
4 1.0400575399398804
5 1.038750171661377
6 1.0394936800003052
7 1.0368696451187134
8 1.0385363101959229
9 1.0386464595794678
10 1.0385699272155762
11 1.0373868942260742
12 1.039320707321167
13 1.0358452796936035
14 1.0391814708709717
15 1.0417566299438477
16 1.0425159931182861
17 1.0377397537231445
18 1.0362887382507324
19 1.039093255996704
20 1.0377638339996338
21 1.039401888847351
22 1.0379372835159302
23 1.0360465049743652
24 1.0382509231567383
25 1.038987398147583
26 1.035846471786499
27 1.037123441696167
28 1.0364514589309692
29 1.040478229522705
30 1.0402435064315796
31 1.0404132604599
32 1.035097360610962
33 1.0392743349075317
34 1.0352579355239868
35 1.0367975234985352
36 1.038354754447937
37 1.0366054773330688
38 1.0370579957962036
39 1.0384496450424194
40 1.0364608764648438
41 1.0399394035339355
42 1.0344599485397339
43 1.0372461080551147
44 1.0371848344802856
45 1.0367157459259033
46 1.03837108

KeyboardInterrupt: 